<a href="https://colab.research.google.com/github/RezhaTaufik/TensorFlow-Titanic-Data/blob/main/TitanicData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

Linear Regression : A method to predict the next value of data sets / for assumption

Y = mx + b
M = slope

Linear regression allows us to predict stuff irl as well, like life expectancy, grades score, etc.
As long as our data lines correlate, we can predict the outcome of the next value

dftrain.head() // Shows data in a nice way 
Head will show the first 5 data in our dataset

Categorical Data : Any data thats not numerical and we have to turn them into numbers somehow

.unique() // allows us to see what kind of values there are in the dataset
For example for genders there are 2 uniques : Male and Female

Epochs : How many times the model will see the same data
When epochs see the data for the first time, it creates a line of best fit thats not great, this is where epochs comes in.

Feed the model with the same data but with different order, therefore, it will slowly generate a better line of best fit 

Input function is how we want the data to be broken down, like what are the criteria and etc

Batch_size = how many elements will we give while the data is being trained


In [ ]:
dftrain = pd.read_csv ('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # Training Data
dfeval = pd.read_csv ('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # Testing Data
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

CATEGORICAL_COLUMNS = ['sex','n_siblings_spouses', 'parch','class','deck','embark_town','alone']

NUMERIC_COLUMNS = ['age','fare']

feature_columns = []

for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [ ]:
def make_input_fn(data_df, label_df,num_epochs =10,shuffle = True,batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle :
      ds = ds.shuffle(1000) # Randomize order data
    ds = ds.batch(batch_size).repeat(num_epochs) # membagi data menjadi beberapa batch 
    return ds # return batch data set
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn) # train data
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])


0.74242425


In [ ]:
result = list(linear_est.predict(eval_input_fn)) # List the data
print(dfeval.loc[3]) # memberikan data penumpang tertentu
print(y_eval.loc[3]) # apakah mereka hidup atau tidak berdasarkan data asli 
print(result[3]['probabilities'][1]) # [3] berarti data ketiga pada dataset, [0] dan [1] kemungkinan mereka hidup atau tidak


sex                        female
age                          55.0
n_siblings_spouses              0
parch                           0
fare                         16.0
class                      Second
deck                      unknown
embark_town           Southampton
alone                           y
Name: 3, dtype: object
1
0.65075994
